In [ ]:
#%pip install protobuf==3.20.1
%pip install transformers[torch]
%pip install -q sentencepiece
%pip install datasets==2.13.1
%pip install evaluate
%pip install rouge_score
#%pip install git+https://github.com/huggingface/nlp.git@fix-bad-type-in-overflow-check

In [ ]:
QPATH = "Quantlet/4-qode2desc"

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

import os
#if IN_COLAB:
#  os.chdir(f'/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}')

In [3]:
import pickle
import json
import re
import sys
from IPython.display import display
import datetime

from tqdm import tqdm
import pandas as pd
import numpy as np
tqdm.pandas()


import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import  DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline
from transformers import AdamW
from datasets import load_dataset

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

import nltk
nltk.download('punkt')
import evaluate

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
encoder_max_length = 512
decoder_max_length = 150
RS = 42
LR = 5e-5

EPOCHS = 10
TRAIN_BATCH = 16
EVAL_BATCH  = 4

WARMUP_STEPS  = 500
WEIGHT_DECAY  = 0.1
LOGGING_STEPS = 100
SAVE_TOTAL_LIM = 1
SAVE_STRATEGY = 'steps'

LABEL_SMOOTHING  = 0.1
PREDICT_GENERATE = True

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model = AutoModelWithLMHead.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, skip_special_tokens=False)
model.to(device)
print(device)

In [ ]:
#train.to_csv('../../data/preprocessed/Quantlet/train_df_with_domain_200231002.csv', index=False)
#val.to_csv('../../data/preprocessed/Quantlet/val_df_with_domain_200231002.csv', index=False)
test = pd.read_csv('../../data/preprocessed/Quantlet/test_df_with_domain_20231002.csv')

In [ ]:
test_dataset_json = {'version' : '10.1',
                    'data' : [{'input_sequence'  : test['code_script'].iloc[i],
                              'output_sequence' : test['Description'].iloc[i]} for i in range(test.shape[0])]}

with open('test.json', 'w') as f:
  json.dump(test_dataset_json, f)

In [ ]:
test_dataset = load_dataset("json",
                            data_files='test.json',
                            field="data",
                            data_dir='./')

In [ ]:
validation_data_txt = test_dataset['train']

In [ ]:
validation_data = validation_data_txt.map(
        lambda batch: analysis_modules.batch_tokenize_preprocess(
            batch,
            tokenizer=tokenizer,
            max_input_length=encoder_max_length,
            max_output_length=decoder_max_length
        ),
        batched=True,
        remove_columns=validation_data_txt.column_names,
    )

In [ ]:
def generate_summary(test_samples, model, tokenizer, encoder_max_length):
    inputs = tokenizer(
        test_samples["input_sequence"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [ ]:
summaries_before_tuning = generate_summary(validation_data_txt,
                                            model,
                                            tokenizer,
                                            encoder_max_length)[1]

In [ ]:


    validation_data_txt = test_dataset['train']

    train_data = train_data_txt.map(
        lambda batch: batch_tokenize_preprocess(
            batch,
            tokenizer=tokenizer,
            max_input_length=encoder_max_length,
            max_output_length=decoder_max_length
        ),
        batch_size=8,
        batched=True,
        remove_columns=train_data_txt.column_names,
    )

    validation_data = validation_data_txt.map(
        lambda batch: batch_tokenize_preprocess(
            batch,
            tokenizer=tokenizer,
            max_input_length=encoder_max_length,
            max_output_length=decoder_max_length
        ),
        batched=True,
        remove_columns=validation_data_txt.column_names,
    )


    # SUBSAMPLE FOR GENERATION BEFORE TUNING
    test_samples = validation_data_txt.select(range(20))


In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
print(analysis_name)
analysis_modules.scs_analyze(analysis_name=analysis_name,
                                    model_name=model_name,
                                    train_data_path=f'../../data/preprocessed/Quantlet/{MODE}/',
                                    train_data_name='full_train_dataset_20230923.json',
                                    val_data_path=f'../../data/preprocessed/Quantlet/{MODE}/',
                                    val_data_name='test_dataset_20230923.json',
                                    encoder_max_length=encoder_max_length,
                                    decoder_max_length=decoder_max_length,
                                    random_state=RS,
                                    eval_columns_list=EVAL_COLUMNS,
                                    learning_rate=LR,
                                    epochs=EPOCHS,
                                    train_batch=TRAIN_BATCH,
                                    eval_batch=EVAL_BATCH,
                                    warmup_steps=WARMUP_STEPS,
                                    weight_decay=WEIGHT_DECAY,
                                    logging_stes=LOGGING_STEPS,
                                    save_total_lim=SAVE_TOTAL_LIM,
                                    save_strategy=SAVE_STRATEGY,
                                    label_smooting=LABEL_SMOOTHING,
                                    predict_generate=PREDICT_GENERATE,
                                    load_best_model_at_end=load_best_model_at_end)